# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
print(db.list_colletions_names())

In [ ]:
# review the collections in our new database


In [ ]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

In [ ]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': "Restaurant/Cafe/Canteen"}
ids = {"BusinessTypeId": 1, 'BusinessType': 1}

# Capture the results in a a variable
results = establishments.find(query, ids)

for result in results:
    pprint(result)


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"BusinessName":"Penang Flavours"},
                      {'$set':{"BusinessTypeId": 1}})

In [ ]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
db.establishments.count_documents({"LocalAuthorityName": "Dover"})

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
establishments.delete_many(query)
results = establishments.find(query)
for result in results:
    pprint(result)

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
db.establishments.count_documents({"LocalAuthorityName": "Dover"})

In [ ]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
results = establishments.find(query)
for result in results:
    pprint(result)

In [ ]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
query = {}
latitude = [{"$set": {"geocode.latitude":{"$toDecimal":"$geocode.latitude"}}}]
longitude = [{"$set": {"geocode.longitude":{"$toDecimal":"$geocode.longitude"}}}]
establishments.update_many(query,latitude)
establishments.update_many(query,longitude)

In [ ]:
#Check to see if change in latitude occurred
lat_results = establishments.find({'BusinessType': 'Restaurant/Cafe/Canteen'},{"geocode.latitude":{"$type": "$geocode.latitude"}})
for x in lat_results:
    print(x)

In [ ]:
#Check to see if change in longitude occurred
long_results = establishments.find({'BusinessType': 'Restaurant/Cafe/Canteen'},{"geocode.longitude":{"$type": "$geocode.longitude"}})
for x in long_results:
    print(x)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
# Check that the coordinates and rating value are now numbers
